## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2025">The Repo</a>.  

Once you have updated your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://github.com/apps/databricks">Databricks App on Github</a> or by cloning the repo to your laptop and then uploading the final_project directory and its contents to your workspace using file imports.  Your choice.

<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches to triggering your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)
- [In class examples - Spark Structured Streaming Performance](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/2638424645880316?o=1093580174577663)

### Be sure your project runs end to end when *Run all* is executued on this notebook! (7 points)

### This project is worth 25% of your final grade.
- DSCC-202 Students have 55 possible points on this project (see points above and the instructions below)
- DSCC-402 Students have 60 possible points on this project (one extra section to complete)

In [0]:
 %run ./includes/includes


In [0]:
#  %run ./includes/utilities

In [0]:
# I PULLED START TIME here, becouse the includes initializing is taking time:

# Set the notebooks starting time.
START_TIME = time.time()

### ADJUST THE MODEL BATCH SIZE:


In [0]:
batch_size=64
hf_sentiment=set_hf_sentiment_model_batch_size(batch_size=batch_size)

### A quick test run, to check that my prediction code is running..

In [0]:
# %run ./includes/test

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

#Additional widget:

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    # Optimize the tables
    optimize_table(BRONZE_DELTA)
    optimize_table(SILVER_DELTA)
    optimize_table(GOLD_DELTA)
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here (2 points)
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

### ANSWER 1) 
a) Below I checked my partitions and cores that I have. It is recommended to set partitions two times
 the cores. 

b) Librarires  are added in the **LIBRARIES** cell in **[utilities](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/4468159549031945?o=1093580174577663)/**

In [0]:
#Current Partitions as default:
spark.conf.get("spark.sql.shuffle.partitions")


In [0]:
#How many core
sc.defaultParallelism


In [0]:
#Then setting it to 8, which is recommended for now having x2 of cores:
spark.conf.set("spark.sql.shuffle.partitions", 8)
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

## 2.0 Define and execute utility functions (3 points)
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files


### ANSWER 2) Cells below:

In [0]:
# ENTER YOUR CODE HERE
#A) Read the source file directory listing

files = dbutils.fs.ls(TWEET_SOURCE_PATH)

In [0]:
# display(dbutils.fs.ls("dbfs:/FileStore/tables/"))

In [0]:
print('The folder looks like this:')
display(files[:2])
source_file_count = len(files)
print("*"*40)
print(f"B) There are {source_file_count} source files in the source directory.")
print("*"*40)
print("C) Sample Json looks like this:")
sample_path=files[0].path
sample_df = spark.read.json(sample_path)
display(sample_df)
print('-Columns in in Json file:',sample_df.columns)

## TEST DATA FILE: 
I created a test sample set for quick checks if code was running. It is such that it copies file to the test folder under new names so that stream keeps reading and writing...
- Functions are added into the [UTILITIES/](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/4468159549031945?o=1093580174577663)
- * copy_new_test_files(num_files=10)  
- create a temp folder under USER_DIR defined
- copy most recent n files into that folder 
- use these folder fur initial test runs

#### TEST FILE:
 This is just a sample run of the preditcion model. A toy data and a test file for the HF MODEL  in ./incldues/test

In [0]:
# %run ./includes/test

In [0]:
#Uncomment this box for test runs..

# dbutils.fs.rm(TEST_PATH, recurse=True)
# copy_new_test_files(num_files=10)

# print('The folder looks like this:')
# print(f'Test folder has {len(dbutils.fs.ls(TEST_PATH))} ')
# test_files=spark.createDataFrame(dbutils.fs.ls(TEST_PATH))
# display(test_files.select('name').distinct().count())

In [0]:
spark.streams.active

## 3.0 Transform the Raw Data to Bronze Data using a stream  (8 points)
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using delta lake to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defined in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook


### ANSWER 3:
- I added the stream set-up code under [UTILITIES/](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/4468159549031945?o=1093580174577663) folder into the **BRONZE_STREAM** cell. 
- it also stops current streams if any before starting the reading. Function call

 *bronze_restart(source_path=TEST_PATH, delay_between_starts=5)*

*!! Don't forget to select your FILE SOURCE:  TEST or MAIN TWEETS folder in the cell below.




In [0]:
# bronze_restart(source_path=TEST_PATH, delay_between_starts=5)
bronze_query=bronze_restart(source_path=TWEET_SOURCE_PATH, delay_between_starts=5)


In [0]:
# copy_new_test_files(num_files=20)

In [0]:
#checking active streams:
for stream in spark.streams.active:
    print(f"Name: {stream.name}")
    print(f"Is Active: {stream.isActive}")
    print(f"Status: {stream.status}")
    # print(stream.recentProgress)
    print("------")



## 4.0 Transform the Bronze Data to Silver Data using a stream (5 points)
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

### ANSWER 4:
- I added the stream set-up code under [UTILITIES/](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/4468159549031945?o=1093580174577663) folder into the SILVER_STREAM cell. Function call:
*silver_restart(delay_between_starts=5)*

- it also stops current streams if any.

In [0]:
silver_query=silver_restart(delay_between_starts=5)

## 5.0 Transform the Silver Data to Gold Data using a stream (7 points)
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

#### ASNWER 5:
-  I added the stream set-up code under [UTILITIES/](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/4468159549031945?o=1093580174577663) folder into the GOLD_STREAM cell. Function call:
*gold_restart(delay_between_starts=5)*

- it also stops current streams if any.

- Before running gold I will try different batch sizes to further check the performance:




In [0]:

gold_query=gold_restart(delay_between_starts=5)


In [0]:
print(gold_query.name)
print(gold_query.status)

## 6.0 Monitor your Streams (5 points)

- Setup a loop that runs at least every 10 seconds
- Print a timestamp of the monitoring query along with the list of streams, rows processed on each, and the processing time on each
- Run the loop until all of the data is processed (0 rows read on each active stream)
- Plot a line graph that shows the data processed by each stream over time
- Plot a line graph that shows the average processing time on each stream over time

### ANSWER 6

- Functions are added in the **MONITORING and PLOTTING ** cells in **[utilities](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/4468159549031945?o=1093580174577663)/** 

- Cell 6.a) monitor_streams()
- Cell 6.b) plot_stream_metrics()

In [0]:
for s in spark.streams.active:
    print(s.name)
    print(s.status)

In [0]:
progress_log = []
idle_rounds = {}
IDLE_LIMIT = 3

while True:
    latest_metrics = monitor_streams(interval_sec=10)
    progress_log.extend(latest_metrics)

    # 2️⃣ Print a live summary table
    if latest_metrics:
        print("\n---- Batch Summary -----------------------------------")
        for m in latest_metrics:
            ts = m["timestamp"]
            query = m["query"]
            rows = m["input_rows"]
            proc = m["processing_time"] or "N/A"
            print(f"{ts} | {query:<20} rows={rows:<5} proc_ms={proc}")
    else:
        print("⏳ No progress from streams yet...")

    # 3️⃣ Track idle streaks
    for m in latest_metrics:
        q = m["query"]
        idle_rounds[q] = idle_rounds.get(q, 0) + (1 if m["input_rows"] == 0 else -idle_rounds.get(q, 0))

    # 4️⃣ Check if all streams idle
    if spark.streams.active and all(idle_rounds.get(q.name, 0) >= IDLE_LIMIT for q in spark.streams.active):
        print("\n✅ All active streams have been idle for "
              f"{IDLE_LIMIT} consecutive polls — monitoring loop exiting.")
        break

    if not spark.streams.active:
        print("\n✅ No active streams — monitoring loop exiting.")
        break

# Plot updated stream metrics:
plot_stream_metrics(progress_log)

### **LOADING DATA FILES:**



In [0]:
# Apply IDLE CHECK AND TURN OFF:

gold_log = stream_until_idle(gold_query, idle_rounds_required=3, interval_sec=10)


In [0]:
gold_query.awaitTermination()

In [0]:

bronze_df = load_bronze_data()
print(f"Total tweets in Bronze: {bronze_df.count()}")
print('______________________')
bronze_df.printSchema()


silver_df = load_silver_data()
print(f"Total tweets in Silver: {silver_df.count()}")
print('______________________')
silver_df.printSchema()





gold_df = load_gold_data()
print(f"Total tweets in Gold: {gold_df.count()}")
print('______________________')
gold_df.printSchema()



## 7.0 Bronze Data Exploratory Data Analysis (5 points)




### ANSWER 7:

I have the cells below to compute answers to these questions.. Will paste some numerical answers back here.

- How many tweets are captured in your Bronze Table? 

- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this? 

- * **I have filltered out NAN values and empty tweet cells ('Cleaned_Text") in my Gold Data, since the model does not except Nan Values.Actually what I fillteres is what is left after @Mention removals!!**

- Count the number of tweets by each unique user handle and sort the data by descending count.

- How many tweets have at least one mention (@) how many tweet have no mentions (@)

- Plot a bar chart that shows the top 20 tweeters (users)

In [0]:
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

# 1. How many tweets are captured in your Bronze Table?
total_tweets = bronze_df.count()
print(f"Total tweets in Bronze: {total_tweets}")

# 2. Null/NaN/empty counts per column via filter (no F. prefix)
for c in bronze_df.columns:
    cnt = bronze_df.filter(
        col(c).isNull() | (col(c) == "")
    ).count()
    print(f"Column `{c}` → {cnt} null/empty values")


# 3. Count tweets per user, descending
user_counts = (
    bronze_df
      .groupBy("user")
      .count()
      .orderBy(desc("count"))
)
user_counts.show(20, truncate=False)   # top 20

# 4. Mentions: how many tweets have at least one “@” vs none
mention_stats = (
    bronze_df
      .withColumn("has_mention", when(col("text").contains("@"), 1).otherwise(0))
      .groupBy("has_mention")
      .count()
      .orderBy("has_mention")
)
mention_stats.show()

# 5. Plot top 20 tweeters
top20 = user_counts.limit(20).toPandas()

plt.figure(figsize=(12, 8))
plt.barh(top20["user"], top20["count"])
plt.gca().invert_yaxis()  # highest at top
plt.xlabel("Number of Tweets")
plt.title("Top 20 Tweeters by Tweet Count")
plt.tight_layout()
plt.show()


In [0]:
#how many dropped just for having mentions in the tweets:

(bronze_df.withColumn("cleaned_text", regexp_replace(col("text"), "@\\w+", ""))
      .withColumn("trimmed", trim(col("cleaned_text")))
      .filter(col("trimmed") == "")
      .count())

In [0]:
#After removing tweets, length of tweets chars :

(bronze_df.withColumn("cleaned_text", regexp_replace(col("text"), "@\\w+", ""))
      .withColumn("text_len", length(trim(col("cleaned_text"))))
      .groupBy("text_len").count()
      .orderBy("text_len")
      .show(10))

## 8.0 Capture the accuracy metrics from the gold table in MLflow  (4 points)
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the model name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

### ANSWER 8:

- I have added the MlFlow setting in the **ML_Flow Registry cell in**  **[utilities](https://dbc-f85bdc5b-07db.cloud.databricks.com/editor/notebooks/4468159549031945?o=1093580174577663)/**. The function to call:
- * mlflow_run_calls()
- here I will assign the results to  **y_pred** and **y_true** from the gold_df and run..

In [0]:
#to make sure Gold Table has finised processing the tweets:
gold_query.awaitTermination()

In [0]:

gold_pd = (
    gold_df
      .select("sentiment_id", "predicted_sentiment_id")
      .toPandas()
)

# Now slice out two Series
y_true = gold_pd["sentiment_id"]
y_pred = gold_pd["predicted_sentiment_id"]



In [0]:
endtime = time.time()
elapsed_time = endtime - START_TIME
print(f"Total run time: {timedelta(seconds=elapsed_time)}")

In [0]:

#Call MlFLow :
mlflow_run_calls(
    y_true, y_pred,
    model_name=HF_MODEL_NAME,
    silver_delta_path=SILVER_DELTA,
    run_name=f'batch_size={batch_size}/Partition={spark.conf.get("spark.sql.shuffle.partitions")}',
    extra_params={"batch_size": batch_size, "elapsed_time": timedelta(seconds=elapsed_time)}
   )


## 9.0 Application Data Processing and Visualization (6 points)
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
gold_df.printSchema()

In [0]:

# 1. Count sentiment label frequency per mention
mention_sentiment_df = (
    gold_df
    .filter(col("mention").isNotNull())
    .groupBy("mention")
    .pivot("predicted_sentiment")  # should be "POS", "NEG", "NEU"
    .count()
    .fillna(0)
)

# 2. Add total column and sort
sentiment_cols = [c for c in mention_sentiment_df.columns if c != "mention"]
mention_counts_df = (
    mention_sentiment_df
    .withColumn("total", expr(" + ".join(f"`{c}`" for c in sentiment_cols)))
    .orderBy(col("total").desc())
)

# 3. Total number of mentions
total_mentions = gold_df.filter(col("mention").isNotNull()).count()
print(f"✅ Total mention occurrences in Gold: {total_mentions}")

# 4. Show top 20 mentions by overall activity
display(mention_counts_df.limit(20))


In [0]:
#Positive sentiments mentions:

if "POS" in mention_counts_df.columns:
    pos_pd = (
        mention_counts_df
        .select("mention", "POS")
        .orderBy(col("POS").desc())
        .limit(20)
        .toPandas()
    )

    plt.figure(figsize=(10, 6))
    plt.barh(pos_pd["mention"], pos_pd["POS"])
    plt.gca().invert_yaxis()
    plt.xlabel("Positive Tweet Count")
    plt.title("Top 20 Mentions by Positive Sentiment")
    plt.tight_layout()
    plt.show()


In [0]:
#negative sentiments top list:
if "NEG" in mention_counts_df.columns:
    neg_pd = (
        mention_counts_df
        .select("mention", "NEG")
        .orderBy(col("NEG").desc())
        .limit(20)
        .toPandas()
    )

    plt.figure(figsize=(10, 6))
    plt.barh(neg_pd["mention"], neg_pd["NEG"])
    plt.gca().invert_yaxis()
    plt.xlabel("Negative Tweet Count")
    plt.title("Top 20 Mentions by Negative Sentiment")
    plt.tight_layout()
    plt.show()


## 10.0 Clean up and completion of your pipeline (3 points)
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook. Note: In the includes there is a variable START_TIME that captures the starting time of the notebook.


### ANSWER 10:
- In the Monitoring and Plotting section above, **(see ANSWER 6)** I have stopped the streams after checking for idle time.



In [0]:

from datetime import timedelta
#current streams:
for stream in spark.streams.active:
    print(f"🔄 Stream: {stream.name} | IsActive: {stream.isActive}")

#stop them:
stop_all_streams()

# Timing the notebook:
elapsed_seconds = time.time() - START_TIME
print(f"⏱️ Notebook Elapsed Time: {str(timedelta(seconds=elapsed_seconds))}")

In [0]:
active_streams = spark.streams.active

if not active_streams:
    print("✅ No active streams.")
else:
    for s in active_streams:
        print(s.name or "(Unnamed Stream)")
        print("------")

## 11.0 How Optimized is your Spark Application (Grad Students Only) (5 points)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ANSWER 11:
I have written a report in a Google Doc which has the screenshots and comments. There I have three sections 
- **Optimization Criteria:** Answer to this section Optimization questions
- **MLFlow Sreenshots:** What I have seen in the MlFlow, the experiments I have done so far. Can be couple more when submitted.
- **In this Project:** I wanted to summarize what I learned in this project.

The link to the Google Docs is here: [Click here for the document report](https://docs.google.com/document/d/1j6ssN2TYuTTPMCIjDhtigyfV9iKQuKUV9lCAWDWB4T0/edit?usp=sharing)